In [1]:
import numpy as np
import pandas as pd

In [26]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2007.xlsx", encoding='utf8')

In [27]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,18h10,Sábado,12/05/2007,Figueirense,Atlético-PR,3x6,Atlético-PR,1ª Rodada,Orlando Scarpelli
1,18h10,Sábado,12/05/2007,São Paulo,Goiás,2x0,São Paulo,1ª Rodada,Morumbi
2,18h10,Sábado,12/05/2007,Fluminense,Cruzeiro,2x2,-,1ª Rodada,Maracanã
3,16h00,Domingo,13/05/2007,Flamengo,Palmeiras,2x4,Palmeiras,1ª Rodada,Maracanã
4,16h00,Domingo,13/05/2007,Internacional,Botafogo,2x3,Botafogo,1ª Rodada,Beira Rio


** Get all clubes for championship **



In [28]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [29]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [30]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,figueirense
1,são paulo
2,fluminense
3,flamengo
4,internacional
5,paraná
6,atlético-mg
7,corinthians
8,américa-rn
9,sport


In [31]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [32]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,figueirense,53.0,38.0,14.0,11.0,13.0,57.0,56.0,1.0
1,são paulo,77.0,38.0,23.0,8.0,7.0,55.0,19.0,36.0
2,fluminense,61.0,38.0,16.0,13.0,9.0,57.0,39.0,18.0
3,flamengo,61.0,38.0,17.0,10.0,11.0,55.0,49.0,6.0
4,internacional,54.0,38.0,15.0,9.0,14.0,49.0,44.0,5.0
5,paraná,41.0,38.0,11.0,8.0,19.0,42.0,64.0,-22.0
6,atlético-mg,55.0,38.0,15.0,10.0,13.0,63.0,51.0,12.0
7,corinthians,44.0,38.0,10.0,14.0,14.0,40.0,50.0,-10.0
8,américa-rn,17.0,38.0,4.0,5.0,29.0,24.0,80.0,-56.0
9,sport,51.0,38.0,14.0,9.0,15.0,54.0,55.0,-1.0


In [33]:
reorderlist = [ 'São Paulo', 'Santos', 'Flamengo', 'Fluminense', 'Cruzeiro', 'Grêmio', 'Palmeiras', 'Atlético-mg', 
                'Botafogo', 'Vasco', 'Internacional', 'Atlético-pr', 'Figueirense', 'Sport', 'Náutico', 'Goiás', 
                'Corinthians', 'Juventude', 'Paraná', 'América-rn']
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            break
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,são paulo,77.0,38.0,23.0,8.0,7.0,55.0,19.0,36.0
2,santos,62.0,38.0,19.0,5.0,14.0,57.0,47.0,10.0
3,flamengo,61.0,38.0,17.0,10.0,11.0,55.0,49.0,6.0
4,fluminense,61.0,38.0,16.0,13.0,9.0,57.0,39.0,18.0
5,cruzeiro,60.0,38.0,18.0,6.0,14.0,73.0,58.0,15.0
6,grêmio,58.0,38.0,17.0,7.0,14.0,44.0,43.0,1.0
7,palmeiras,58.0,38.0,16.0,10.0,12.0,48.0,47.0,1.0
8,atlético-mg,55.0,38.0,15.0,10.0,13.0,63.0,51.0,12.0
9,botafogo,55.0,38.0,14.0,13.0,11.0,62.0,58.0,4.0
10,vasco,54.0,38.0,15.0,9.0,14.0,58.0,47.0,11.0
